In [1]:
!pip uninstall -y protobuf tensorflow-metadata
!pip install protobuf==3.20.3 tensorflow-metadata==1.13.1


Found existing installation: protobuf 6.33.0
Uninstalling protobuf-6.33.0:
  Successfully uninstalled protobuf-6.33.0
Found existing installation: tensorflow-metadata 1.17.2
Uninstalling tensorflow-metadata-1.17.2:
  Successfully uninstalled tensorflow-metadata-1.17.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 4.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 3.20.3 which is incompatible.
onnx 1.18.0 requires protobuf>=4.25.1, but you have protobuf 3.20.3 which is incompatible.
a2a-sdk 0.3.10 requires protobuf>=5.29.5, but you have protobuf 3.20.3 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.

# imports

In [2]:
import tensorflow as tf
import numpy as np
import os


2025-12-15 14:05:11.504598: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765807511.681514      20 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765807511.739160      20 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# config

In [3]:
BASE_DIR = "/kaggle/working/mepe"
DATA_DIR = f"{BASE_DIR}/data/fer2013"
MODEL_DIR = f"{BASE_DIR}/models/face_emotion"

IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 10
LR = 1e-4
NUM_CLASSES = 7


# adding FER2013 Dataset

In [4]:
!ls /kaggle/input

facial-emotion-recognition-vgg19-fer2013  fer2013
facial-expression-recognition-using-cnn   fer2013-with-keras


In [5]:
# first we added the dataset to the input, by using add input
!mkdir -p /kaggle/working/mepe/data


In [6]:
!cp -r /kaggle/input/fer2013 /kaggle/working/mepe/data/


In [7]:
!ls /kaggle/working/mepe/data/


fer2013


In [8]:
!ls /kaggle/working/mepe/data/fer2013


fer2013.csv


we found the right datadet, now it was in csv. we have to create the folder structure(train,teat&val) and then convert the .csv into images & add them to their respective folders

In [9]:
import os

BASE_DIR = "/kaggle/working/mepe/data/fer2013"
splits = ["train", "val", "test"]
emotions = ["angry","disgust","fear","happy","sad","surprise","neutral"]

for split in splits:
    for emo in emotions:
        os.makedirs(f"{BASE_DIR}/{split}/{emo}", exist_ok=True)

print("Folders created")


Folders created


In [10]:
import pandas as pd

df = pd.read_csv("/kaggle/input/fer2013/fer2013.csv")
df.head()


,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [11]:
label_map = {
    0: "angry",
    1: "disgust",
    2: "fear",
    3: "happy",
    4: "sad",
    5: "surprise",
    6: "neutral"
}

usage_map = {
    "Training": "train",
    "PublicTest": "val",
    "PrivateTest": "test"
}


In [12]:
# convert csv -> images

import numpy as np
import cv2
from tqdm import tqdm

for idx, row in tqdm(df.iterrows(), total=len(df)):
    emotion = label_map[row["emotion"]]
    split = usage_map[row["Usage"]]

    pixels = np.array(row["pixels"].split(), dtype="uint8").reshape(48, 48)
    img_path = f"{BASE_DIR}/{split}/{emotion}/{idx}.png"

    cv2.imwrite(img_path, pixels)


100%|██████████| 35887/35887 [00:19<00:00, 1848.14it/s]


In [13]:
!ls /kaggle/working/mepe/data/fer2013/train/happy | head


10000.png
10001.png
10005.png
10019.png
10023.png
10035.png
10039.png
10042.png
10046.png
1004.png
ls: write error: Broken pipe


# Load FER2013

In [14]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    f"{DATA_DIR}/train",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="categorical"
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    f"{DATA_DIR}/val",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="categorical"
)

class_names = train_ds.class_names
class_names


Found 28709 files belonging to 7 classes.


I0000 00:00:1765807554.745496      20 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1765807554.746109      20 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Found 3589 files belonging to 7 classes.


['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

# performance pipeline

In [15]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.prefetch(AUTOTUNE)
val_ds = val_ds.prefetch(AUTOTUNE)


# Build Face Emotion Model

In [16]:
base_model = tf.keras.applications.ResNet50(
    weights="imagenet",
    include_top=False,
    input_shape=(224, 224, 3)
)

base_model.trainable = False  # phase-3: frozen backbone

inputs = tf.keras.Input(shape=(224, 224, 3))
x = tf.keras.applications.resnet.preprocess_input(inputs)
x = base_model(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)

face_embedding = tf.keras.layers.Dense(
    256, activation="relu", name="face_embedding"
)(x)

emotion_logits = tf.keras.layers.Dense(
    NUM_CLASSES, activation="softmax", name="emotion_logits"
)(face_embedding)

model = tf.keras.Model(inputs, emotion_logits)
model.summary()


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item (GetItem)  │ (None, 224, 224)  │          0 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_1          │ (None, 224, 224)  │          0 │ input_layer_1[0]… │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_2          │ (None, 224, 224)  │          0 │ input_layer_1[0]… │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stack (Stack)       │ (None, 224, 224,  │          0 │ get_item[0][0],   │
│                     │ 3)                │            │ get_item_1[0][0], │
│                     │                   │            │ get_item_2[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 224, 224,  │          0 │ stack[0][0]       │
│                     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ resnet50            │ (None, 7, 7,      │ 23,587,712 │ add[0][0]         │
│ (Functional)        │ 2048)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 2048)      │          0 │ resnet50[0][0]    │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ face_embedding      │ (None, 256)       │    524,544 │ global_average_p… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ emotion_logits      │ (None, 7)         │      1,799 │ face_embedding[0… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 24,114,055 (91.99 MB)

 Trainable params: 526,343 (2.01 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

> # compile

In [17]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(LR),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)


# Train

In [18]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS
)


Epoch 1/10


I0000 00:00:1765807569.549332      88 service.cc:148] XLA service 0x7ff9a4016140 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1765807569.550122      88 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1765807569.550149      88 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1765807571.068878      88 cuda_dnn.cc:529] Loaded cuDNN version 90300


  1/898 ━━━━━━━━━━━━━━━━━━━━ 4:06:30 16s/step - accuracy: 0.0312 - loss: 2.8661

I0000 00:00:1765807575.665826      88 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


898/898 ━━━━━━━━━━━━━━━━━━━━ 103s 96ms/step - accuracy: 0.3981 - loss: 1.5772 - val_accuracy: 0.5007 - val_loss: 1.3284
Epoch 2/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 83s 93ms/step - accuracy: 0.5319 - loss: 1.2497 - val_accuracy: 0.5336 - val_loss: 1.2516
Epoch 3/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 89s 100ms/step - accuracy: 0.5730 - loss: 1.1597 - val_accuracy: 0.5489 - val_loss: 1.2080
Epoch 4/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 94s 105ms/step - accuracy: 0.5971 - loss: 1.0963 - val_accuracy: 0.5667 - val_loss: 1.1807
Epoch 5/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 94s 105ms/step - accuracy: 0.6167 - loss: 1.0473 - val_accuracy: 0.5662 - val_loss: 1.1833
Epoch 6/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 95s 105ms/step - accuracy: 0.6324 - loss: 1.0057 - val_accuracy: 0.5623 - val_loss: 1.1629
Epoch 7/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 94s 104ms/step - accuracy: 0.6512 - loss: 0.9607 - val_accuracy: 0.5815 - val_loss: 1.1473
Epoch 8/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 94s 105ms/step - accuracy: 0.6667 - loss: 0.9290 - val_

# save model

In [19]:
import os

FACE_DIR = "/kaggle/working/mepe/models/face_emotion"
os.makedirs(FACE_DIR, exist_ok=True)

model.save(f"{FACE_DIR}/model.keras")
print("Face emotion model saved")


Face emotion model saved


# Extract embedding model ( for fusion )

In [20]:
embedding_model = tf.keras.Model(
    inputs=model.input,
    outputs=model.get_layer("face_embedding").output
)

embedding_model.summary()
# this model does:
# (face image) --> (256-dim representation)

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item (GetItem)  │ (None, 224, 224)  │          0 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_1          │ (None, 224, 224)  │          0 │ input_layer_1[0]… │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_2          │ (None, 224, 224)  │          0 │ input_layer_1[0]… │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stack (Stack)       │ (None, 224, 224,  │          0 │ get_item[0][0],   │
│                     │ 3)                │            │ get_item_1[0][0], │
│                     │                   │            │ get_item_2[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 224, 224,  │          0 │ stack[0][0]       │
│                     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ resnet50            │ (None, 7, 7,      │ 23,587,712 │ add[0][0]         │
│ (Functional)        │ 2048)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 2048)      │          0 │ resnet50[0][0]    │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ face_embedding      │ (None, 256)       │    524,544 │ global_average_p… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 24,112,256 (91.98 MB)

 Trainable params: 524,544 (2.00 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

# Inference test

In [21]:
for images, labels in val_ds.take(1):
    emb = embedding_model(images[:1])
    print("Embedding shape:", emb.shape)


Embedding shape: (1, 256)


# saving class names (for inference later )

In [22]:
import os
import json

FACE_DIR = "/kaggle/working/mepe/models/face_emotion"
os.makedirs(FACE_DIR, exist_ok=True)

with open(f"{FACE_DIR}/classes.json", "w") as f:
    json.dump(class_names, f)

print("classes.json saved")


classes.json saved


# Save embedding only model

In [23]:
embedding_model.save("/kaggle/working/mepe/models/face_embedding.keras")
print("embedding model was saved")
# this step is not much needed

embedding model was saved


# saving the model in tf format

In [24]:
FACE_HF_DIR = "/kaggle/working/mepe/models/face_emotion_hf"

os.makedirs(FACE_HF_DIR, exist_ok=True)
model.save(f"{FACE_HF_DIR}/model.keras")

print("Face emotion model saved")


Face emotion model saved


# verifying the saved one

In [25]:
import os
print(os.listdir("/kaggle/working/mepe/models/face_emotion"))


['model.keras', 'classes.json']


# downloading the notebook

In [26]:
!zip -r face_emotion.zip /kaggle/working/mepe/models/face_emotion


  adding: kaggle/working/mepe/models/face_emotion/ (stored 0%)
  adding: kaggle/working/mepe/models/face_emotion/model.keras (deflated 8%)
  adding: kaggle/working/mepe/models/face_emotion/classes.json (deflated 18%)
